In [1]:
import os 

In [2]:
%pwd 

'c:\\Users\\Nyos\\Documents\\text-sum\\textsummarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd 

'c:\\Users\\Nyos\\Documents\\text-sum\\textsummarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path 
    #output_dir='pegasus-samsum', 
    num_train_epochs: int 
    warmup_steps: int
    per_device_train_batch_size: int
    #per_device_eval_batch_size: 1
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str 
    eval_steps: int
    save_steps: float 
    gradient_accumulation_steps: int 

In [6]:
from textsummarizer.const import *
from textsummarizer.utility.commons import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            #output_dir='pegasus-samsum', 
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            #per_device_eval_batch_size: 1
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy  = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps  
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer 
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch 
import os

c:\Users\Nyos\anaconda3\envs\textsum\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-30 21:39:56,524: INFO: config: PyTorch version 2.2.1 available.]


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config 

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        """ trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        ) """

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, 
            num_train_epochs = 1, 
            warmup_steps = 500,
            per_device_train_batch_size= 1,
            #per_device_eval_batch_size: 1
            weight_decay = 0.01,
            logging_steps = 10,
            evaluation_strategy = 'steps',
            eval_steps = 500,
            save_steps = 1e6,
            gradient_accumulation_steps = 16
        )


        trainer = Trainer(model=model_pegasus, args=trainer_args,
                          tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["test"],
                          eval_dataset=dataset_samsum_pt["validation"])
        trainer.train() 

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e 

[2024-03-30 21:39:57,128: INFO: commons: yaml file: config\config.yaml loaded successfully]
[2024-03-30 21:39:57,135: INFO: commons: yaml file: params.yaml loaded successfully]
[2024-03-30 21:39:57,137: INFO: commons: created directory at: artifacts]
[2024-03-30 21:39:57,140: INFO: commons: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Nyos\anaconda3\envs\textsum\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
 20%|█▉        | 10/51 [31:51<2:10:10, 190.49s/it]

{'loss': 3.1101, 'grad_norm': 31.9448184967041, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [1:03:43<1:40:39, 194.82s/it]

{'loss': 3.0466, 'grad_norm': 15.821706771850586, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

{'loss': 2.9898, 'grad_norm': 19.90555191040039, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}


 98%|█████████▊| 50/51 [2:41:45<03:11, 191.70s/it]

{'loss': 2.8523, 'grad_norm': 33.813507080078125, 'learning_rate': 5e-06, 'epoch': 0.98}


100%|██████████| 51/51 [2:44:46<00:00, 193.85s/it]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


{'train_runtime': 9886.1678, 'train_samples_per_second': 0.083, 'train_steps_per_second': 0.005, 'train_loss': 3.0227226416269937, 'epoch': 1.0}
